In [7]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import re

base_results_folder = "../1-archive-runs/07-novel-run"  # folder containing scenario subfolders
cv = False


def smooth(data, window=10):
    if window > 1 and len(data) >= window:
        return np.convolve(data, np.ones(window)/window, mode='valid')
    return data

# Scan the base folder for scenario subdirectories
scenario_folders = [
    f for f in os.listdir(base_results_folder) 
    if os.path.isdir(os.path.join(base_results_folder, f))
]

sns.set(style="darkgrid")

for scenario in scenario_folders:
    # Path to the numpy folder inside this scenario
    numpy_dir = os.path.join(base_results_folder, scenario, "numpy")
    if not os.path.exists(numpy_dir):
        # If there's no numpy folder, skip this scenario
        continue
    
    # Load the arrays
    try:
        all_myopic_runs = np.load(os.path.join(numpy_dir, "all_myopic_runs.npy"))
        all_myopic_steps_runs = np.load(os.path.join(numpy_dir, "all_myopic_steps_runs.npy"))
        all_proactive_runs = np.load(os.path.join(numpy_dir, "all_proactive_runs.npy"))
        all_proactive_steps_runs = np.load(os.path.join(numpy_dir, "all_proactive_steps_runs.npy"))
        all_reactive_runs = np.load(os.path.join(numpy_dir, "all_reactive_runs.npy"))
        all_reactive_steps_runs = np.load(os.path.join(numpy_dir, "all_reactive_steps_runs.npy"))
        
        # Test rewards - check individual seed files instead of aggregated files
        test_rewards_myopic = []
        test_rewards_proactive = []
        test_rewards_reactive = []
        
        # Get list of seeds from file names
        seed_pattern = re.compile(r'test_rewards_myopic_seed_(\d+)\.npy')
        seed_files = [f for f in os.listdir(numpy_dir) if seed_pattern.match(f)]
        seeds = [seed_pattern.match(f).group(1) for f in seed_files]
        
        for seed in seeds:
            myopic_path = os.path.join(numpy_dir, f"test_rewards_myopic_seed_{seed}.npy")
            proactive_path = os.path.join(numpy_dir, f"test_rewards_proactive_seed_{seed}.npy")
            reactive_path = os.path.join(numpy_dir, f"test_rewards_reactive_seed_{seed}.npy")
            
            if os.path.exists(myopic_path):
                test_rewards_myopic.append(np.load(myopic_path).tolist())
            if os.path.exists(proactive_path):
                test_rewards_proactive.append(np.load(proactive_path).tolist())
            if os.path.exists(reactive_path):
                test_rewards_reactive.append(np.load(reactive_path).tolist())

        # Convert to numpy arrays if any data was loaded and pad sequences to same length
        if test_rewards_myopic:
            max_len = max(len(x) for x in test_rewards_myopic)
            test_rewards_myopic = np.array([x + [np.nan]*(max_len-len(x)) for x in test_rewards_myopic])
        if test_rewards_proactive:
            max_len = max(len(x) for x in test_rewards_proactive)
            test_rewards_proactive = np.array([x + [np.nan]*(max_len-len(x)) for x in test_rewards_proactive])
        if test_rewards_reactive:
            max_len = max(len(x) for x in test_rewards_reactive)
            test_rewards_reactive = np.array([x + [np.nan]*(max_len-len(x)) for x in test_rewards_reactive])

    except FileNotFoundError as e:
        print(f"Missing files in {numpy_dir}, skipping scenario '{scenario}'. Error: {e}")
        continue

    # Compute means and std
    myopic_mean = all_myopic_runs.mean(axis=0)
    myopic_std = all_myopic_runs.std(axis=0)
    proactive_mean = all_proactive_runs.mean(axis=0)
    proactive_std = all_proactive_runs.std(axis=0)
    reactive_mean = all_reactive_runs.mean(axis=0)
    reactive_std = all_reactive_runs.std(axis=0)

    myopic_steps_mean = all_myopic_steps_runs.mean(axis=0).astype(int)
    proactive_steps_mean = all_proactive_steps_runs.mean(axis=0).astype(int)
    reactive_steps_mean = all_reactive_steps_runs.mean(axis=0).astype(int)

    # Smooth data for plotting 
    smooth_window = 1
    
    myopic_mean_sm = smooth(myopic_mean, smooth_window)
    myopic_std_sm = smooth(myopic_std, smooth_window)
    myopic_steps_sm = myopic_steps_mean[:len(myopic_mean_sm)]

    proactive_mean_sm = smooth(proactive_mean, smooth_window)
    proactive_std_sm = smooth(proactive_std, smooth_window)
    proactive_steps_sm = proactive_steps_mean[:len(proactive_mean_sm)]

    reactive_mean_sm = smooth(reactive_mean, smooth_window)
    reactive_std_sm = smooth(reactive_std, smooth_window)
    reactive_steps_sm = reactive_steps_mean[:len(reactive_mean_sm)]

    # Create training plot
    plt.figure(figsize=(14, 6))
    plt.plot(proactive_steps_sm, proactive_mean_sm, label="DQN Proactive-U", color='orange')
    plt.fill_between(proactive_steps_sm, proactive_mean_sm - proactive_std_sm, proactive_mean_sm + proactive_std_sm, alpha=0.2, color='orange')
    plt.plot(myopic_steps_sm, myopic_mean_sm, label="DQN Proactive-N", color='blue')
    plt.fill_between(myopic_steps_sm, myopic_mean_sm - myopic_std_sm, myopic_mean_sm + myopic_std_sm, alpha=0.2, color='blue')
    plt.plot(reactive_steps_sm, reactive_mean_sm, label="DQN Reactive", color='green')
    plt.fill_between(reactive_steps_sm, reactive_mean_sm - reactive_std_sm, reactive_mean_sm + reactive_std_sm, alpha=0.2, color='green')
    plt.xlabel("Environment Steps")
    plt.ylabel("Episode Reward")
    plt.title("Training Performance")
    plt.legend()
    plt.grid(True)
    plt.show()


    if cv:
        # Create testing plot
        plt.figure(figsize=(14, 6))
        if test_rewards_proactive is not None:
            test_mean_proactive = np.nanmean(test_rewards_proactive, axis=0)
            test_std_proactive = np.nanstd(test_rewards_proactive, axis=0)
            plt.plot(test_mean_proactive, label="DQN Proactive-U", color='orange')
            plt.fill_between(range(len(test_mean_proactive)), test_mean_proactive - test_std_proactive, test_mean_proactive + test_std_proactive, alpha=0.2, color='orange')
        
        if test_rewards_myopic is not None:
            test_mean_myopic = np.nanmean(test_rewards_myopic, axis=0)
            test_std_myopic = np.nanstd(test_rewards_myopic, axis=0)
            plt.plot(test_mean_myopic, label="DQN Proactive-N", color='blue')
            plt.fill_between(range(len(test_mean_myopic)), test_mean_myopic - test_std_myopic, test_mean_myopic + test_std_myopic, alpha=0.2, color='blue')
        
        if test_rewards_reactive is not None:
            test_mean_reactive = np.nanmean(test_rewards_reactive, axis=0)
            test_std_reactive = np.nanstd(test_rewards_reactive, axis=0)
            plt.plot(test_mean_reactive, label="DQN Reactive", color='green')
            plt.fill_between(range(len(test_mean_reactive)), test_mean_reactive - test_std_reactive, test_mean_reactive + test_std_reactive, alpha=0.2, color='green')
        
        plt.xlabel("Test Episode")
        plt.ylabel("Episode Reward")
        plt.title("Testing Performance")
        plt.legend()
        plt.grid(True)
        plt.show()

    # # Combined plot
    # ax3.plot(proactive_steps_sm, proactive_mean_sm, label="Train DQN Proactive-U", color='orange')
    # ax3.fill_between(proactive_steps_sm, proactive_mean_sm - proactive_std_sm, proactive_mean_sm + proactive_std_sm, alpha=0.2, color='orange')
    # ax3.plot(myopic_steps_sm, myopic_mean_sm, label="Train DQN Proactive-N", color='blue')
    # ax3.fill_between(myopic_steps_sm, myopic_mean_sm - myopic_std_sm, myopic_mean_sm + myopic_std_sm, alpha=0.2, color='blue')
    # ax3.plot(reactive_steps_sm, reactive_mean_sm, label="Train DQN Reactive", color='green')
    # ax3.fill_between(reactive_steps_sm, reactive_mean_sm - reactive_std_sm, reactive_mean_sm + reactive_std_sm, alpha=0.2, color='green')

    # # Add test points
    # if test_rewards_proactive is not None:
    #     ax3.plot(proactive_steps_sm[-1], test_mean_proactive[-1], 'o', label="Test DQN Proactive-U", color='orange', markersize=10)
    # if test_rewards_myopic is not None:
    #     ax3.plot(myopic_steps_sm[-1], test_mean_myopic[-1], 'o', label="Test DQN Proactive-N", color='blue', markersize=10)
    # if test_rewards_reactive is not None:
    #     ax3.plot(reactive_steps_sm[-1], test_mean_reactive[-1], 'o', label="Test DQN Reactive", color='green', markersize=10)

    # ax3.set_xlabel("Environment Steps")
    # ax3.set_ylabel("Episode Reward")
    # ax3.set_title("Combined Training and Testing Performance")
    # ax3.legend()
    # ax3.grid(True)

    # plt.tight_layout()
    # plt.show()


In [8]:
# Load and print config values from config.csv
config_df = pd.read_csv(base_results_folder + "/config.csv")
for column in config_df.columns:
    print(f"{column}= {config_df[column].iloc[0]}")


np= <module 'numpy' from 'C:\\Users\\Admin\\Desktop\\Boeing-ADM-DRL-Github\\.venv\\Lib\\site-packages\\numpy\\__init__.py'>
MAX_AIRCRAFT= 6
MAX_FLIGHTS_PER_AIRCRAFT= 20
ROWS_STATE_SPACE= 7
COLUMNS_STATE_SPACE= 63
ACTION_SPACE_SIZE= 147
DEPARTURE_AFTER_END_RECOVERY= 1
TIMESTEP_HOURS= 1
DUMMY_VALUE= -999
RESOLVED_CONFLICT_REWARD= 5000
DELAY_MINUTE_PENALTY= 11.5
MAX_DELAY_PENALTY= 2500
NO_ACTION_PENALTY= 0
CANCELLED_FLIGHT_PENALTY= 5000
LAST_MINUTE_THRESHOLD= 120
LAST_MINUTE_FLIGHT_PENALTY= 455
AHEAD_BONUS_PER_MINUTE= 0.1
TIME_MINUTE_PENALTY= 1
TERMINATION_REWARD= 500
MIN_TURN_TIME= 0
MIN_BREAKDOWN_PROBABILITY= 0
DEBUG_MODE= False
DEBUG_MODE_TRAINING= False
DEBUG_MODE_REWARD= False
DEBUG_MODE_PRINT_STATE= False
DEBUG_MODE_CANCELLED_FLIGHT= False
DEBUG_MODE_VISUALIZATION= False
DEBUG_MODE_BREAKDOWN= False
DEBUG_MODE_ACTION= False
DEBUG_MODE_STOPPING_CRITERIA= False
DEBUG_MODE_SCHEDULING= False
DEBUG_MODE_REWARD_LAST_MINUTE_PENALTY= False
DEBUG_MODE_REWARD_RESOLVED_CONFLICTS= False
MAX_TOTA

In [9]:
import sys
sys.path.append("..")
import os
import numpy as np
import pandas as pd
from stable_baselines3 import DQN
import torch
import time
import re
import matplotlib.pyplot as plt
from scipy.stats import ttest_rel

from src.environment import AircraftDisruptionEnv
from scripts.visualizations import StatePlotter
from scripts.utils import load_scenario_data
from src.config import *

def run_inference_dqn(model_path, scenario_folder, env_type, seed):
    # Load scenario data
    data_dict = load_scenario_data(scenario_folder)
    aircraft_dict = data_dict['aircraft']
    flights_dict = data_dict['flights']
    rotations_dict = data_dict['rotations']
    alt_aircraft_dict = data_dict['alt_aircraft']
    config_dict = data_dict['config']

    # Initialize environment
    env = AircraftDisruptionEnv(
        aircraft_dict, 
        flights_dict, 
        rotations_dict, 
        alt_aircraft_dict, 
        config_dict,
        env_type=env_type
    )

    # Load model
    model = DQN.load(model_path)
    model.set_env(env)
    model.policy.set_training_mode(False)
    model.exploration_rate = 0.0

    np.random.seed(seed)
    torch.manual_seed(seed)

    # Reset the environment
    obs, _ = env.reset()
    done_flag = False
    total_reward = 0
    step_num = 0
    max_steps = 1000

    # Start timing the inference
    start_time = time.time()

    while not done_flag and step_num < max_steps:
        # Convert observation to float32
        obs = {key: np.array(value, dtype=np.float32) for key, value in obs.items()}

        # Get action mask
        action_mask = obs.get('action_mask', None)
        if action_mask is None:
            raise ValueError("Action mask is missing in the observation!")

        # Get Q-values
        obs_tensor = model.policy.obs_to_tensor(obs)[0]
        q_values = model.policy.q_net(obs_tensor).detach().cpu().numpy().squeeze()

        # Mask invalid actions
        masked_q_values = q_values.copy()
        masked_q_values[action_mask == 0] = -np.inf

        action = np.argmax(masked_q_values)

        # Step environment
        obs, reward, terminated, truncated, info = env.step(action)
        total_reward += reward
        done_flag = terminated or truncated
        step_num += 1

    # End timing
    end_time = time.time()
    runtime = end_time - start_time

    return total_reward, step_num, runtime

# Example seed and training IDs
seed = 26
training_id_myopic = 506
training_id_proactive = 513
training_id_reactive = 518

model_to_compare = [
    f"../trained_models/dqn/6ac-350-diverse/{seed}/training_{training_id_myopic}/myopic-training_{training_id_myopic}.zip",
    f"../trained_models/dqn/6ac-350-diverse/{seed}/training_{training_id_proactive}/proactive-training_{training_id_proactive}.zip",
    f"../trained_models/dqn/6ac-350-diverse/{seed}/training_{training_id_reactive}/reactive-training_{training_id_reactive}.zip",
]

# Define the scenario folder set
SCENARIO_FOLDER = "../data/Testing/6ac-350-diverse/"

# Extract all scenario folders
scenario_folders = [os.path.join(SCENARIO_FOLDER, d) for d in os.listdir(SCENARIO_FOLDER) 
                    if os.path.isdir(os.path.join(SCENARIO_FOLDER, d))]
scenario_folders.sort()

def extract_env_type(model_path):
    match = re.search(r'/(myopic|proactive|reactive)', model_path)
    if not match:
        raise ValueError(f"Model path {model_path} does not contain a valid environment type!")
    return match.group(1)

# Number of runs
N_RUNS = 5

num_scenarios = len(scenario_folders)
num_models = len(model_to_compare)

results_all_runs = np.zeros((N_RUNS, num_scenarios, num_models))
runtime_all_runs = np.zeros((N_RUNS, num_scenarios, num_models))

base_seed = 12345

for run_idx in range(N_RUNS):
    current_seed = base_seed + run_idx
    
    for m_idx, model_path in enumerate(model_to_compare):
        if not os.path.exists(model_path):
            raise FileNotFoundError(f"Model file not found: {model_path}")
            
        env_type = extract_env_type(model_path)
        
        for s_idx, scenario_folder in enumerate(scenario_folders):
            if not os.path.exists(scenario_folder):
                raise FileNotFoundError(f"Scenario folder not found: {scenario_folder}")
            
            total_reward, steps, runtime = run_inference_dqn(model_path, scenario_folder, env_type, current_seed)
            # Store results
            results_all_runs[run_idx, s_idx, m_idx] = total_reward
            runtime_all_runs[run_idx, s_idx, m_idx] = runtime

# Compute run-level means for rewards and runtime
run_mean_rewards = np.mean(results_all_runs, axis=1)  # shape: (N_RUNS, num_models)
run_mean_runtime = np.mean(runtime_all_runs, axis=1)  # shape: (N_RUNS, num_models)

# Compute final mean and std across runs
final_mean_rewards = np.mean(run_mean_rewards, axis=0)
final_std_rewards = np.std(run_mean_rewards, axis=0)

final_mean_runtime = np.mean(run_mean_runtime, axis=0)
final_std_runtime = np.std(run_mean_runtime, axis=0)

model_names = [os.path.basename(m) for m in model_to_compare]

# Print final results
print("Final Average Rewards (averaged over scenarios and then over runs):")
for m_name, val in zip(model_names, final_mean_rewards):
    print(f"{m_name}: {val}")

print("\nFinal Std of Rewards (across runs):")
for m_name, val in zip(model_names, final_std_rewards):
    print(f"{m_name}: {val}")

print("\nFinal Average Runtime (averaged over scenarios and then over runs):")
for m_name, val in zip(model_names, final_mean_runtime):
    print(f"{m_name}: {val}")

print("\nFinal Std of Runtime (across runs):")
for m_name, val in zip(model_names, final_std_runtime):
    print(f"{m_name}: {val}")

# Pairwise t-tests for rewards
print("\nPairwise p-values for REWARDS:")
p_value_rewards = pd.DataFrame(index=model_names, columns=model_names)
for i in range(num_models):
    for j in range(num_models):
        if i == j:
            p_value_rewards.iloc[i, j] = np.nan
        else:
            stat, p_val = ttest_rel(run_mean_rewards[:, i], run_mean_rewards[:, j])
            p_value_rewards.iloc[i, j] = p_val
print(p_value_rewards)

# Pairwise t-tests for runtime
print("\nPairwise p-values for RUNTIME:")
p_value_runtime = pd.DataFrame(index=model_names, columns=model_names)
for i in range(num_models):
    for j in range(num_models):
        if i == j:
            p_value_runtime.iloc[i, j] = np.nan
        else:
            stat, p_val = ttest_rel(run_mean_runtime[:, i], run_mean_runtime[:, j])
            p_value_runtime.iloc[i, j] = p_val
print(p_value_runtime)

# Plotting using the actual calculated values
labels = model_names
bar_colors = ['orange', 'blue', 'green'][:len(labels)]  # In case you have fewer or more models

y = np.arange(len(labels))  # the label locations



FileNotFoundError: [Errno 2] No such file or directory: '../data/Testing/6ac-350-diverse/'

In [ ]:


# Plot rewards
plt.figure(figsize=(7, 2))
plt.barh(y, final_mean_rewards, xerr=final_std_rewards, capsize=5, color=bar_colors, alpha=0.3)
plt.xlabel('Mean Cumulative Reward', fontsize=12)
plt.yticks(y, labels, fontsize=12)
plt.axvline(x=0, color='gray', linewidth=1, linestyle='--')
plt.title('Mean Cumulative Reward by DQN Approach')
plt.grid(alpha=0)
plt.tight_layout()
plt.show()

# Plot runtime
plt.figure(figsize=(7, 2))
plt.barh(y, final_mean_runtime, xerr=final_std_runtime, capsize=5, color=bar_colors, alpha=0.3)
plt.xlabel('Mean Runtime (seconds)', fontsize=12)
plt.yticks(y, labels, fontsize=12)
plt.title('Mean Runtime by DQN Approach')
plt.grid(alpha=0)
plt.tight_layout()
plt.show()